# 1.0 Loading Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
 

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

# 2.0 Loading Dataset


In [2]:
df_raw = pd.read_csv("dataset.csv")
df_raw = df_raw.rename(columns={"Attrition": "Resignation"})
df_raw.head()

,Age,Resignation,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


# 3.0 Split into train and test data

I'm splitting the dataset at this point because that way the test data will simulate the production data, that is, never seen data.

In [3]:
X = df_raw.drop('Resignation', axis = 1)
y = df_raw[['Resignation']]

In [4]:
test_size = 0.3
seed = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = seed)

In [5]:
print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_test: {y_test.shape}')

X_train: (1029, 34)
y_train: (1029, 1)
X_test: (441, 34)
y_test: (441, 1)


In [6]:
df_raw = pd.concat([X_train, y_train], axis=1)

# 4.0 Descriptive Analisys

In [7]:
df_raw.shape

(1029, 35)

In [8]:
df_raw.iloc[:, 0:17].info()

<class 'pandas.core.frame.DataFrame'>
Index: 1029 entries, 714 to 1126
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Age                      1029 non-null   int64 
 1   BusinessTravel           1029 non-null   object
 2   DailyRate                1029 non-null   int64 
 3   Department               1029 non-null   object
 4   DistanceFromHome         1029 non-null   int64 
 5   Education                1029 non-null   int64 
 6   EducationField           1029 non-null   object
 7   EmployeeCount            1029 non-null   int64 
 8   EmployeeNumber           1029 non-null   int64 
 9   EnvironmentSatisfaction  1029 non-null   int64 
 10  Gender                   1029 non-null   object
 11  HourlyRate               1029 non-null   int64 
 12  JobInvolvement           1029 non-null   int64 
 13  JobLevel                 1029 non-null   int64 
 14  JobRole                  1029 non-null   ob

In [9]:
df_raw.iloc[:, 17:].info()

<class 'pandas.core.frame.DataFrame'>
Index: 1029 entries, 714 to 1126
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   MonthlyIncome             1029 non-null   int64 
 1   MonthlyRate               1029 non-null   int64 
 2   NumCompaniesWorked        1029 non-null   int64 
 3   Over18                    1029 non-null   object
 4   OverTime                  1029 non-null   object
 5   PercentSalaryHike         1029 non-null   int64 
 6   PerformanceRating         1029 non-null   int64 
 7   RelationshipSatisfaction  1029 non-null   int64 
 8   StandardHours             1029 non-null   int64 
 9   StockOptionLevel          1029 non-null   int64 
 10  TotalWorkingYears         1029 non-null   int64 
 11  TrainingTimesLastYear     1029 non-null   int64 
 12  WorkLifeBalance           1029 non-null   int64 
 13  YearsAtCompany            1029 non-null   int64 
 14  YearsInCurrentRole        1

In [10]:
df_raw.iloc[:, :17].nunique().sort_values(ascending=False)

EmployeeNumber             1029
DailyRate                   710
HourlyRate                   71
Age                          43
DistanceFromHome             29
JobRole                       9
EducationField                6
Education                     5
JobLevel                      5
EnvironmentSatisfaction       4
JobInvolvement                4
JobSatisfaction               4
BusinessTravel                3
Department                    3
MaritalStatus                 3
Gender                        2
EmployeeCount                 1
dtype: int64

In [11]:
df_raw.iloc[:, 17:].nunique().sort_values(ascending=False)

MonthlyRate                 1010
MonthlyIncome                963
TotalWorkingYears             40
YearsAtCompany                32
YearsInCurrentRole            19
YearsWithCurrManager          17
YearsSinceLastPromotion       16
PercentSalaryHike             15
NumCompaniesWorked            10
TrainingTimesLastYear          7
WorkLifeBalance                4
StockOptionLevel               4
RelationshipSatisfaction       4
PerformanceRating              2
OverTime                       2
Resignation                    2
StandardHours                  1
Over18                         1
dtype: int64

I'll exclude all features that have only 1 distinct value because these features have variance = 0 and that's bad for the model

In [12]:
df_raw_numeric = df_raw.select_dtypes('int64')

kurt = pd.DataFrame(df_raw_numeric.apply(lambda x: x.kurtosis())).T
skew = pd.DataFrame(df_raw_numeric.apply(lambda x: x.skew())).T
variance = pd.DataFrame(df_raw_numeric.var().round(2)).applymap(lambda x: "{:.{}f}".format(x, 2) if isinstance(x, (float)) else x).T
desc = pd.DataFrame(df_raw_numeric.describe())

df_stats = pd.concat([desc,  kurt, skew, variance]).T.reset_index()
df_stats.columns = ['atributos','count', 'mean', 'std','min', '25%', '50%', '75%', 'max', 'kurtosis', 'skew', 'variance']
df_stats

,atributos,count,mean,std,min,25%,50%,75%,max,kurtosis,skew,variance
0,Age,1029.0,36.910593,9.263531,18.0,30.0,35.0,43.0,60.0,-0.471263,0.403126,85.81
1,DailyRate,1029.0,802.620019,407.368993,102.0,464.0,804.0,1167.0,1496.0,-1.218684,-0.005694,165949.50
2,DistanceFromHome,1029.0,9.364431,8.222723,1.0,2.0,7.0,14.0,29.0,-0.30753,0.931019,67.61
3,Education,1029.0,2.892128,1.053541,1.0,2.0,3.0,4.0,5.0,-0.648794,-0.273762,1.11
4,EmployeeCount,1029.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.00
5,EmployeeNumber,1029.0,1024.367347,606.301635,1.0,496.0,1019.0,1553.0,2068.0,-1.20654,0.021478,367601.67
6,EnvironmentSatisfaction,1029.0,2.683188,1.096829,1.0,2.0,3.0,4.0,4.0,-1.242343,-0.265805,1.20
7,HourlyRate,1029.0,66.680272,20.474094,30.0,48.0,67.0,84.0,100.0,-1.212044,-0.088747,419.19
8,JobInvolvement,1029.0,2.713314,0.710146,1.0,2.0,3.0,3.0,4.0,0.2552,-0.503142,0.50
9,JobLevel,1029.0,2.043732,1.118918,1.0,1.0,2.0,3.0,5.0,0.322603,1.024197,1.25


The features appear to have a distribution close to the normal distribution. Almost all features have skewness around 0, which may indicate that the data is not very concentrated.

# 5.0 EDA - Exploratory Data Analisys

In [13]:
df_eda = df_raw.copy()

In [14]:
print(f"The dataset is highly unbalanced")
print(f"Class No (Not Resgign): {round(df_eda.groupby('Resignation').size()[0] / df_eda.shape[0] * 100, 2)}%")
print(f"Class Yes (Resgign): {round(df_eda.groupby('Resignation').size()[1] / df_eda.shape[0] * 100, 2)}%")


The dataset is highly unbalanced
Class No (Not Resgign): 82.9%
Class Yes (Resgign): 17.1%


In [19]:
color_by_class = {
    "Yes": "#4E2F2F",
    "No": "#31688E"
}


fig = px.histogram(df_eda, x="Resignation", color="Resignation", color_discrete_map=color_by_class)
fig.update_layout(
    title="Resignation Distribution",
    xaxis_title="Class",
    yaxis_title="Number of employees",
    height=400, 
    width=600 
)
fig.show()

It's possible to notice that there is an unbalanced class. More than 82% of the employees didn't resign.

In [35]:
features_to_exclude_zero_one_value = df_raw.nunique() == 1
features_to_exclude_zero_one_value = df_raw.loc[:, features_to_exclude_zero_one_value].columns
features_to_exclude_zero_one_value

Index(['EmployeeCount', 'Over18', 'StandardHours'], dtype='object')